In [ ]:
# ============================================================
# ⚙️ INSTALLATION ET IMPORTS
# ============================================================
!pip install torch torchvision tqdm -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import cv2, numpy as np, pickle
from google.colab import drive

# ============================================================
# 1️⃣ MONTAGE DU DRIVE
# ============================================================
drive.mount("/content/drive", force_remount=True)

# ============================================================
# 2️⃣ DATASET PICKLE (inchangé)
# ============================================================
class PKLVideoDataset(Dataset):
    def __init__(self, x_files, y_files, transform=None, target_frames=16):
        self.videos, self.labels = [], []
        self.transform = transform
        self.target_frames = target_frames
        for xf, yf in zip(x_files, y_files):
            with open(xf, "rb") as f: self.videos.extend(pickle.load(f))
            with open(yf, "rb") as f: self.labels.extend(pickle.load(f))

    def __len__(self): return len(self.videos)

    def __getitem__(self, idx):
        video, label = self.videos[idx], self.labels[idx]
        frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in video]

        # Normaliser la longueur des vidéos
        if len(frames) < self.target_frames:
            repeat = int(np.ceil(self.target_frames / len(frames)))
            frames = (frames * repeat)[:self.target_frames]
        else:
            frames = frames[:self.target_frames]

        if self.transform:
            frames = [self.transform(f) for f in frames]
        video_tensor = torch.stack(frames, dim=1)  # (C, T, H, W)
        return video_tensor, torch.tensor(label, dtype=torch.long)

# ============================================================
# 3️⃣ TRANSFORMATIONS (spécifique à MViTv2)
# ============================================================
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # entrée attendue 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                         std=[0.22803, 0.22145, 0.216989]),
])

# ============================================================
# 4️⃣ CHEMINS DES DONNÉES
# ============================================================
x_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train8_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train7_small3.pkl",

]
y_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train8.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train7.pkl",
]
# Fichiers de validation
x_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val8_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val7_small3.pkl",

]
y_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val8.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val7.pkl",

]
# ============================================================
# 5️⃣ DATALOADERS
# ============================================================
train_dataset = PKLVideoDataset(x_files, y_files, transform=transform)
val_dataset = PKLVideoDataset(x_val_files, y_val_files, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

# ============================================================
# 6️⃣ MODÈLE MViTv2
# ============================================================
class MViTv2Custom(nn.Module):
    def __init__(self, num_classes=20, pretrained=True, dropout_p=0.3):
        super().__init__()
        weights = models.video.MViT_V2_S_Weights.KINETICS400_V1 if pretrained else None
        self.backbone = models.video.mvit_v2_s(weights=weights)
        in_features = self.backbone.head[1].in_features
        self.backbone.head[1] = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_p),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)



Mounted at /content/drive


In [ ]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MViTv2Custom(num_classes=20).to(device)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scaler = GradScaler()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

num_epoch = 10
best_acc = 0.0
model_path = "/content/drive/MyDrive/Projet non-alternant/Vidéo/best_mvitvnop.pth"

for epoch in range(num_epoch):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for videos, labels in tqdm(train_loader, desc=f"Époque {epoch+1}/{num_epoch}"):
        videos, labels = videos.to(device), labels.to(device)
        optimizer.zero_grad()



        with autocast():
            outputs = model(videos)
            loss = criterion(outputs, labels)

        # 🔧 Backpropagation avec GradScaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(device), labels.to(device)
            with autocast():
                outputs = model(videos)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    val_loss /= len(val_loader)
    scheduler.step()

    print(f"📊 Époque [{epoch+1}/{num_epoch}] "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | "
          f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"💾 Nouveau meilleur modèle sauvegardé (Val Acc: {best_acc:.4f})")

print("✅ Entraînement terminé. Meilleure précision validation :", best_acc)


Downloading: "https://download.pytorch.org/models/mvit_v2_s-ae3be167.pth" to /root/.cache/torch/hub/checkpoints/mvit_v2_s-ae3be167.pth


100%|██████████| 132M/132M [00:00<00:00, 142MB/s]
/tmp/ipython-input-1814043583.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Époque 1/10:   0%|          | 0/1489 [00:00<?, ?it/s]/tmp/ipython-input-1814043583.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Époque 1/10: 100%|██████████| 1489/1489 [17:07<00:00,  1.45it/s]
/tmp/ipython-input-1814043583.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


📊 Époque [1/10] Train Acc: 0.2986 | Val Acc: 0.4221 | Train Loss: 2.3967 | Val Loss: 2.0438
💾 Nouveau meilleur modèle sauvegardé (Val Acc: 0.4221)


Époque 2/10:   8%|▊         | 119/1489 [01:23<15:52,  1.44it/s]